# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcb2099aa60>
├── 'a' --> tensor([[ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718]])
└── 'x' --> <FastTreeValue 0x7fcb2099aa90>
    └── 'c' --> tensor([[ 0.5873,  0.1504,  0.1417,  0.8499],
                        [-0.7327, -1.2987, -0.6745,  0.3113],
                        [ 0.5979,  0.8779,  1.8151,  1.0372]])

In [4]:
t.a

tensor([[ 0.5037, -0.2940, -0.2364],
        [-0.1660,  0.3115, -2.0718]])

In [5]:
%timeit t.a

66.1 ns ± 0.0654 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcb2099aa60>
├── 'a' --> tensor([[-0.6911,  0.6460, -0.3729],
│                   [ 0.1930, -0.1178,  0.6163]])
└── 'x' --> <FastTreeValue 0x7fcb2099aa90>
    └── 'c' --> tensor([[ 0.5873,  0.1504,  0.1417,  0.8499],
                        [-0.7327, -1.2987, -0.6745,  0.3113],
                        [ 0.5979,  0.8779,  1.8151,  1.0372]])

In [7]:
%timeit t.a = new_value

67.4 ns ± 0.0793 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718]]),
    x: Batch(
           c: tensor([[ 0.5873,  0.1504,  0.1417,  0.8499],
                      [-0.7327, -1.2987, -0.6745,  0.3113],
                      [ 0.5979,  0.8779,  1.8151,  1.0372]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5037, -0.2940, -0.2364],
        [-0.1660,  0.3115, -2.0718]])

In [11]:
%timeit b.a

58.1 ns ± 0.45 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5031, -0.0870,  0.9516],
               [-0.2755, -0.5261, -1.8269]]),
    x: Batch(
           c: tensor([[ 0.5873,  0.1504,  0.1417,  0.8499],
                      [-0.7327, -1.2987, -0.6745,  0.3113],
                      [ 0.5979,  0.8779,  1.8151,  1.0372]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.0187 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

831 ns ± 1.68 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 91.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 413 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 323 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fca74675040>
├── 'a' --> tensor([[[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]],
│           
│                   [[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]],
│           
│                   [[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]],
│           
│                   [[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]],
│           
│                   [[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]],
│           
│                   [[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]],
│           
│                   [[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]],
│           
│                   [[ 0.5037, -0.2940, -0.2364],
│                    [-0.1660,  0.3115, -2.0718]]])
└── 'x' --> <FastTreeValue 0x7fca746752e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 61 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcb209a1370>
├── 'a' --> tensor([[ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718],
│                   [ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718],
│                   [ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718],
│                   [ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718],
│                   [ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718],
│                   [ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718],
│                   [ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718],
│                   [ 0.5037, -0.2940, -0.2364],
│                   [-0.1660,  0.3115, -2.0718]])
└── 'x' --> <FastTreeValue 0x7fcb209a1580>
    └── 'c' --> tensor([[ 0.5873,  0.1504,  0.1417,  0.8499],
                        [-0.7327, -1.2987, -0.6745,  0.3113],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 64 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 78.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]],
       
               [[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]],
       
               [[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]],
       
               [[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]],
       
               [[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]],
       
               [[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]],
       
               [[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]],
       
               [[ 0.5037, -0.2940, -0.2364],
                [-0.1660,  0.3115, -2.0718]]]),
    x: Batch(
           c: tensor([[[ 0.5873,  0.1504,  0.1417,  0.8499],
                       [-0.7327, -1.2987, -0.6745,  0.3113],
                       [ 0.5979,  0.8779,  1.8151,  1.0372]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 885 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718],
               [ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718],
               [ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718],
               [ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718],
               [ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718],
               [ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718],
               [ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718],
               [ 0.5037, -0.2940, -0.2364],
               [-0.1660,  0.3115, -2.0718]]),
    x: Batch(
           c: tensor([[ 0.5873,  0.1504,  0.1417,  0.8499],
                      [-0.7327, -1.2987, -0.6745,  0.3113],
                      [ 0.5979,  0.8779,  1.8151,  1.0372],
                      [ 0.5873,  0.1504,  0.1417,  0.8499],
                      [-0.7327, -

In [28]:
%timeit Batch.cat(batches)

145 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 3.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
